# Accessing data from the Troll Observing Network (TONe) Integrated Cloud Observatory (ICO)

## Use a Dask cluster for higher performance when reading data

- However, this is not essential.

In [1]:
from dask.distributed import Client
client = Client(processes=False)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.0.234:8787/status,
Dashboard: http://192.168.0.234:8787/status,Workers: 1
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: False
Comm: inproc://192.168.0.234/34126/1,Workers: 0
Dashboard: http://192.168.0.234:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: inproc://192.168.0.234/34126/4,Total threads: 8
Dashboard: http://192.168.0.234:57547/status,Memory: 32.00 GiB
Nanny: None,


In [2]:
import icechunk
import pandas as pd
import xarray as xr
import hvplot.xarray

## Define a function that access the Level-0 data repositories

In [3]:
def read_level0_data_repo(repo_L0_bucket):

    repo_prefix = ''
    repo_endpoint_url = 'https://tone-ico-gws-o.s3-ext.jc.rl.ac.uk/'
    repo_region = 'auto'
    repo_anonymous = True

    storage = icechunk.s3_storage(
        bucket=repo_L0_bucket,
        prefix=repo_prefix,
        endpoint_url=repo_endpoint_url,
        region=repo_region,
        anonymous=repo_anonymous,
    )
    repo = icechunk.Repository.open(storage)
    session = repo.readonly_session(branch='main')
    if 'radiosonde' in repo_L0_bucket:
        # ....Radiosonde data are stored as an xarray DataTree
        dt = xr.open_datatree(session.store, engine='zarr', consolidated=True)
        return dt
    else:
        # ....All other TONe-ICO data are stored as xarray DataSets
        ds = xr.open_zarr(session.store, consolidated=False)
        return ds


### Current list of TONe-ICO Level-0 data repos

In [4]:
repo = [
    'automatic-weather-station-troll-station.level0.repo',
    'automatic-weather-station-troll-airfield.level0.repo',
    'depolarisation-lidar-ceilometer.level0.repo',
    'radiosonde.level0.repo',
]

### Read automatic weather station data!

In [5]:
aws_station = read_level0_data_repo(repo[0])
aws_airfield = read_level0_data_repo(repo[1])

In [6]:
aws_station.T_air_2_vent.hvplot() * aws_airfield.T_air_2.hvplot()

:Overlay
   .Curve.I  :Curve   [time]   (T_air_2_vent)
   .Curve.II :Curve   [time]   (T_air_2)

## Read the Lidar data

In [7]:
lidar = read_level0_data_repo(repo[2])
lidar

<xarray.Dataset> Size: 113GB
Dimensions:                            (time: 1714440, range: 3276, layer: 5)
Coordinates:
  * time                               (time) datetime64[ns] 14MB 2025-02-05T...
  * range                              (range) float64 26kB 0.0 ... 1.572e+04
  * layer                              (layer) float64 40B 1.0 2.0 3.0 4.0 5.0
    latitude                           float64 8B ...
    longitude                          float64 8B ...
Data variables: (12/26)
    airplane_filter_max_range          (time) float32 7MB dask.array<chunksize=(5460,), meta=np.ndarray>
    azimuth_angle                      (time) float64 14MB dask.array<chunksize=(5460,), meta=np.ndarray>
    beta_att                           (time, range) float32 22GB dask.array<chunksize=(5460, 3276), meta=np.ndarray>
    cloud_calibration_factor           (time) float64 14MB dask.array<chunksize=(5460,), meta=np.ndarray>
    beta_att_noise_level               (time) float64 14MB dask.array<chunksize=(5460,), meta=np.ndarray>
    cloud_thickness                    (time, layer) float64 69MB dask.array<chunksize=(5460, 5), meta=np.ndarray>
    ...                                 ...
    range_resolution                   (time) float32 7MB dask.array<chunksize=(5460,), meta=np.ndarray>
    p_pol                              (time, range) float32 22GB dask.array<chunksize=(5460, 3276), meta=np.ndarray>
    tilt_angle                         (time) float32 7MB dask.array<chunksize=(5460,), meta=np.ndarray>
    vertical_visibility                (time) float64 14MB dask.array<chunksize=(5460,), meta=np.ndarray>
    x_pol                              (time, range) float32 22GB dask.array<chunksize=(5460, 3276), meta=np.ndarray>
    tilt_correction                    (time) float32 7MB dask.array<chunksize=(5460,), meta=np.ndarray>
Attributes: (12/14)
    title:                          CL61D CL61 with Depolarization
    institution:                    Norwegian Polar Institute
    source:                         tone-ico_depolarisation_lidar_ceilometer
    conventions:                    CF-1.8
    schema_version:                 1.3
    sw_version:                     1.2.7
    ...                             ...
    unit:                           m
    instrument_serial_number:       W2821225
    overlap_function_provided:      1
    overlap_is_corrected:           1
    file_temporal_span_in_minutes:  5.0
    profile_interval_in_seconds:    10

In [8]:
# ....Place the lowest cloud base height into your local computer's memory
#     This speeds up the plotting below !
cbh = lidar.cloud_base_heights[:,0].compute()

In [ ]:
# ....Plot a time series of the lowest cloud base height
cbh.hvplot.scatter()

In [ ]:
# ....Create a violin plot that displays the statistics as a function of height of the lowest cloud base height
cbh.hvplot.violin()

## Read the sonde profiles

- Unfortunately, this takes about 2 minutes, so be PATIENT.

In [ ]:
sonde_profiles = read_level0_data_repo(repo[3])

In [ ]:
# %% - One accesses each level-0 radiosonde profiles like a Python dictionary.
#      Print a list of the profiles, the keys of which are the names of the raw radiosonde *.cor files.

sonde = sonde_profiles.groups
print(sonde[1])

In [ ]:
sonde_profiles[sonde[1]].T.hvplot(y='height')

## Don't forget to close the client to the Dask cluster!

In [ ]:
client.close()